In [1]:
import sys
print(sys.version)

3.10.6 (main, Nov  2 2022, 18:53:38) [GCC 11.3.0]


In [2]:
import sklearn.ensemble
import shap
import numpy as np
import numba
import time
import xgboost
from sklearn.datasets import make_regression

/home/jarek/projects/shap/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
X,y = shap.datasets.boston()
X.shape

Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :func:`~sklearn.datasets.fetch_california_housing`) and the Ames housing
    datase

(506, 13)

In [4]:
X, y = make_regression(n_samples=400, n_features=16, n_informative=8,random_state=0, shuffle=False)

model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000, max_depth=10)
model.fit(X, y)

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [5]:
# bst = xgboost.train({"learning_rate": 0.01, "max_depth": 4}, xgboost.DMatrix(X, label=y), 1000)

In [6]:
# start = time.time()
# shap_values = bst.predict(xgboost.DMatrix(X), pred_contribs=True)
# print(time.time() - start)


In [7]:
from shap.explainers.pytree import *

# x = X.head()
x = X[:1]
# x = X
x

array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ,  1.86755799,
        -0.97727788,  0.95008842, -0.15135721, -0.10321885,  0.4105985 ,
         0.14404357,  1.45427351,  0.76103773,  0.12167502,  0.44386323,
         0.33367433]])

In [8]:
from shap import TreeExplainer as TreeCext
start = time.time()
t = TreeCext(model).shap_values(x, banz=False)
print(time.time() - start)
t

ile drzew cext? 1000
using treeshap and cext
banz = False
here37
using dense_Tree_pred
0.13489651679992676


array([[ 37.23281991,  20.30107456,  30.54561712, 171.23906631,
         57.7996405 , -63.8705223 ,  34.23130434,   4.40778534,
          1.77703487,   0.58573517,   2.65351456,   7.08324677,
          1.72122824,   1.31355515,   0.34448904,   1.37721225]])

In [9]:
start = time.time()
t = TreeCext(model).shap_values(x, banz=True)
print(time.time() - start)
t # TODO BLAD - dodają się zamiast zaczynac od 0 dla kazdej probki danych

ile drzew cext? 1000
using treeshap and cext
banz = True
using cext banz
0.0887906551361084


array([[ 60.18349734,  35.89487379,  29.25075919, 393.905911  ,
         90.24519075, -30.15632353,  52.76623805,  13.44313922,
          3.49392754,  -1.14201064,   3.42015687,   8.6018395 ,
          3.84812707,   2.58968554,   0.95218814,   3.06168016]])

In [10]:
start = time.time()
ex = TreeExplainer(model)
print(time.time() - start)
start = time.time()
print(ex.shap_values(x))
print(time.time() - start)

1.3340599536895752
oryginalna implemenacja shap_values
other model type
using slow python treeshap
[[ 37.23281991  20.30107456  30.54561712 171.23906631  57.7996405
  -63.8705223   34.23130434   4.40778534   1.77703487   0.58573517
    2.65351456   7.08324677   1.72122824   1.31355515   0.34448904
    1.37721225 -10.95613933]]
12.996456623077393


In [11]:
start = time.time()
res = ex.banz_values(x)
print(time.time() - start)
res

nasza implementacja banz values
other model type
wlasciwe obliczenia w sekundzie: 0.023442745208740234
dwa wymiary
typ resa: <class 'list'>, <class 'list'>
4.698397874832153


array([[ 3.51173281e+01,  1.87952565e+01,  2.86345474e+01,
         1.63018360e+02,  5.43228146e+01, -7.03032853e+01,
         3.09257751e+01,  2.97140120e+00,  1.61414093e+00,
         1.59193500e-01,  2.30206194e+00,  5.72311055e+00,
         1.15148742e+00,  1.13902961e+00,  4.10570855e-02,
         1.18671544e+00]])

In [12]:
# start = time.time()
# res = ex.banz_values(x, jit=True)
# print(time.time() - start)
# res

In [13]:
line = np.asarray(x[:1])
ex.brute_banz(line[0], 0)

KeyboardInterrupt: 

In [ ]:
ex.brute_shap(line[0], 0)